## NLP : Identify Good and Bad Comments

In [1]:
import random

class Sentiment:
    Negative="Negative"
    Neutral="Neutral"
    Positive='Positive'
    
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.Negative
        elif self.score==3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment==Sentiment.Negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment==Sentiment.Positive, self.reviews))
        positive_shrink = positive[:len(negative)]
        self.reviews = negative+positive_shrink
        random.shuffle(self.reviews)
        
        #print(negative[0].text)
        print(len(negative))
        print(len(positive))

In [2]:
import json
file_name='./data/sentiment/books_small_10000.json'

reviews=[]
with open(file_name) as f:
    for line in f:
        review= json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data

In [3]:
from sklearn.model_selection import train_test_split

training_set,test_set=train_test_split(reviews,test_size=0.33,random_state=42)

train_container = ReviewContainer(training_set)
test_container = ReviewContainer(test_set)

In [4]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.Positive))
print(train_y.count(Sentiment.Negative))

436
5611
208
2767
436
436


### Bag of words vectorization

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#vectorizer=CountVectorizer()
vectorizer=TfidfVectorizer()
train_x_vec = vectorizer.fit_transform(train_x)
test_x_vec = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vec[0].toarray())
print(train_y[0])

I got half way through and had to quit.  There was nothing I liked about the book.  Oh, I guess the title isn't bad.
[[0. 0. 0. ... 0. 0. 0.]]
Negative


## Classification

#### Linear SVM

In [6]:
from sklearn import svm
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vec,train_y)

print(test_y[0])
clf_svm.predict(test_x_vec[0])

Negative


array(['Negative'], dtype='<U8')

#### Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vec,train_y)

print(test_y[0])
clf_dec.predict(test_x_vec[0])

Negative


array(['Negative'], dtype='<U8')

#### Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()

# fix gaussian issue with dense matrix
dense_train_x_vec=train_x_vec.todense()
dense_test_x_vec=test_x_vec.todense()
clf_gnb.fit(dense_train_x_vec,train_y)

clf_gnb.predict(dense_test_x_vec[0])

array(['Negative'], dtype='<U8')

#### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression()
clf_log.fit(train_x_vec,train_y)

print(test_y[0])
clf_log.predict(test_x_vec[0])

Negative


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array(['Negative'], dtype='<U8')

## Evaluation

In [10]:
# Mean Accuracy
print('svm :',clf_svm.score(test_x_vec,test_y))
print('dec :',clf_dec.score(test_x_vec,test_y))
print('gnb :',clf_gnb.score(dense_test_x_vec,test_y))
print('log :',clf_log.score(test_x_vec,test_y))


svm : 0.8076923076923077
dec : 0.6586538461538461
gnb : 0.6610576923076923
log : 0.8028846153846154


In [11]:
# F1 Scores
from sklearn.metrics import f1_score
print('svm :',f1_score(test_y, clf_svm.predict(test_x_vec), average=None,labels=[Sentiment.Positive,Sentiment.Neutral,Sentiment.Negative]))
print('dec :',f1_score(test_y, clf_dec.predict(test_x_vec), average=None,labels=[Sentiment.Positive,Sentiment.Neutral,Sentiment.Negative]))
print('gnb :',f1_score(test_y, clf_gnb.predict(dense_test_x_vec), average=None,labels=[Sentiment.Positive,Sentiment.Neutral,Sentiment.Negative]))
print('log :',f1_score(test_y, clf_log.predict(test_x_vec), average=None,labels=[Sentiment.Positive,Sentiment.Neutral,Sentiment.Negative]))
# They perform good on positives but shitty on negatives

svm : [0.80582524 0.         0.80952381]
dec : [0.65024631 0.         0.66666667]
gnb : [0.65693431 0.         0.66508314]
log : [0.80097087 0.         0.8047619 ]


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
test_set2=['I thoroughly enjoyed this', 'my grandma loved it','horrible book','shitty book','waste of time','very good book']
new_test = vectorizer.transform(test_set2)

clf_log.predict(new_test)

array(['Positive', 'Positive', 'Positive', 'Positive', 'Negative',
       'Positive'], dtype='<U8')

### Tuning model with Grid Search

In [14]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32)}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vec,train_y)
#tuned_svm = svm.SVC()


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/svm/

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
print('svm :',clf.score(test_x_vec,test_y))


svm : 0.8076923076923077


## Saving Model


In [22]:
import pickle

with open('./models/sentiment_classification.pkl','wb') as f:
    pickle.dump(clf,f)

## Load Model


In [24]:
with open('./models/sentiment_classification.pkl','rb') as f:
    loaded_clf = pickle.load(f)
    
print(test_x[0])

loaded_clf.predict(test_x_vec[0])

I was really annoyed at how the lead female character walked right into trouble constantly.  I was hoping she would have some kind of common sense and strength, but she didn't.  I have no desire to read the next.


array(['Negative'], dtype='<U8')